In [1]:
# %pip install openai -U

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/10/06/691ef3f0112ecf0d7420d0bf35b5d16cf81554141f4b4913a9831031013d/openai-1.55.3-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/1b/0e/1b79afa5616309d4e2e84980c62a3f73c4035e5b856ad7601aebbb5a7db0/jiter-0.8.0-cp312-none-win_amd64.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/d5/74/da832196702d0c56eb86b75bfa346db9238617e29b0b7ee3b8b4eccfe654/pydantic-2.10.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/170.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/170.8 kB ? eta -:--:--
     ------- -----------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
from textwrap import dedent
from openai import OpenAI
from pydantic import BaseModel, Field

from tqdm import tqdm

In [ ]:
api_key = '<api-key>'

In [4]:
client = OpenAI(api_key=api_key)

In [22]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    weather: str
    participants: list[str]

In [25]:
model_str="gpt-4o-2024-11-20"

completion = client.beta.chat.completions.parse(
    model=model_str,
    messages=[
        {"role": "system", "content": "Extract the event information. If you don't know the value, write None"},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

In [26]:
print(completion.choices[0].message.content)

{"name":"Science Fair","date":"Friday","weather":"None","participants":["Alice","Bob"]}


In [18]:
class FundusParameters(BaseModel):
    # Возможно ли проводить офтальмоскопию
    is_valid: bool
    
    # OD
    od_color: str
    od_monotone: str 
    od_size: str
    od_shape: str
    od_border: str
    
    od_excavation_size: str
    od_excavation_location: str
    od_excavation_ratio: str
    od_vessels_location: str
    od_pathology: str
    
    # Vessels
    vessels_art_course: str
    vessels_art_turtuosity: str
    vessels_art_bifurcation: str
    vessels_art_caliber: str
    
    vessels_vein_course: str
    vessels_vein_turtuosity: str
    vessels_vein_bifurcation: str
    vessels_vein_caliber: str
    
    vessels_ratio: str
    vessels_pathology: str
    
    # Macula
    macula_macular_reflex: str
    macula_foveal_reflex: str
    
    # Peripheral
    macula_pathology: str
    peripheral_pathology: str
    
    # Other
    other_diagnosis: str
    other_notes: str

In [41]:
system_prompt_rus = """
Извлеки параметры изображения глазного дна по его описанию. Если значение неизвестно, укажи значение по умолчанию, которое прописано для каждого параметра.
Если проводить офтольмоскопию невозможно, то укажи значение None для всех параметров, кроме is_valid.

Описание параметров:
- is_valid: Возможно ли визуализировать и проводить офтольмоскопию. Значение по по умолчанию - True.
- od_color: Цвет диска зрительного нерва, который может указывать на различные состояния его здоровья (например, серый, бледный или белый). Значение по умолчанию - бледно-розовый.
- od_monotone: Булевый параметр, указывающий, имеет ли диск зрительного нерва монотонный (однородный) цвет. Значение по умолчанию - True.
- od_size: Размер диска зрительного нерва (например, нормальный, больше нормы или меньше нормы), что может помочь выявить отклонения. Значение по умолчанию - нормальный.
- od_shape: Форма диска зрительного нерва (например, правильная, овальная или неправильная). Значение по умолчанию - правильная.
- od_border: Чёткость границ диска зрительного нерва (например, четкие или размытые), важный показатель для диагностики патологий. Значение по умолчанию - четкие.
- od_excavation_size: Размер экскавации (углубления) на диске зрительного нерва (например, нормальный, больше нормы или меньше нормы), который может варьироваться в зависимости от таких состояний, как глаукома. Значение по умолчанию - нормальный.
- od_excavation_location: Расположение экскавации на диске зрительного нерва (например, в центре, верхний, нижний, наружный, внутренний, верхне-наружный, верхне-внутренний, нижне-наружный, нижне-внутренний). Значение по умолчанию - в центре.
- od_excavation_ratio: Отношение размеров экскавации к диску (CDR), ключевой показатель для оценки риска глаукомы. Значение по умолчанию - None.
- od_vessels_location: Расположение сосудистого пучка на диске зрительного нерва (например, в центре, верхний, нижний, наружный, внутренний, верхне-наружный, верхне-внутренний, нижне-наружный, нижне-внутренний). Значение по умолчанию - в центре.
- od_pathology: Свободный текст, описывающий патологию диска зрительного нерва. Значение по умолчанию - None.
- vessels_art_course: Направление или смещение артерий (например, нормальное, смещение наружу или смещение внутрь). Значение по умолчанию - нормальный.
- vessels_art_turtuosity: Извитость артерий (например, нормальная, извитая или прямая), отражающая состояние сосудов. Значение по умолчанию - нормальная.
- vessels_art_bifurcation: Угол бифуркации артерий (например, нормальная, под острым углом или под тупым углом), важный для оценки сосудов. Значение по умолчанию - нормальная.
- vessels_art_caliber: Калибр (ширина) артерий (например, нормальный, расширенный или суженный), который может указывать на проблемы с кровообращением. Значение по умолчанию - нормальный.
- vessels_vein_course: Направление или смещение вен (например, нормальное, наружное или внутреннее смещение). Значение по умолчанию - нормальный.
- vessels_vein_turtuosity: Извитость вен (например, нормальная, извитая или прямая), предоставляющая информацию о системных или глазных заболеваниях. Значение по умолчанию - нормальная.
- vessels_vein_bifurcation: Угол бифуркации вен (например, нормальная, под острым углом или под тупым углом), часто изучается для оценки сосудистых аномалий. Значение по умолчанию - нормальная.
- vessels_vein_caliber: Калибр (ширина) вен (например, нормальный, расширенный или суженный), важный для диагностики сосудистых патологий. Значение по умолчанию - нормальный.
- vessels_ratio: Свободный текст, указывающий соотношение артерий и вен. Значение по умолчанию - None.
- vessels_pathology: Свободный текст, описывающий патологию сосудов. Значение по умолчанию - None.
- macula_macular_reflex: Состояние макулярного рефлекса, который может быть нормальным, гладким или отсутствовать, предоставляя информацию о здоровье макулы. Значение по умолчанию - нормальный.
- macula_foveal_reflex: Состояние фовеального рефлекса (например, нормальный, гладкий или отсутствует), критично для оценки качества центрального зрения. Значение по умолчанию - нормальный.
- macula_pathology: Свободный текст, описывающий патологию макулы. Значение по умолчанию - None.
- peripheral_pathology: Свободный текст, описывающий патологию периферии глазного дна. Значение по умолчанию - None.
- other_diagnosis: Свободный текст, указывающий диагноз. Значение по умолчанию - None.
- other_notes: Свободный текст, указывающий примечания. Значение по умолчанию - None.
"""

Пример с описанием из которого можно выделить параметры

In [30]:
model_str="gpt-4o-2024-11-20"

example_fundus_content = "ДЗН бледно-розовый, монотонный, границы четкие височный сдвиг сосудов, макулярные рефлексы сглажены, на периферии на 6-7 ч ангиоматозный узел, с преретинальным фиброзом, отграниченный лазеркоагулятами"

completion = client.beta.chat.completions.parse(
    model=model_str,
    messages=[
        {"role": "system", "content": dedent(system_prompt_rus)},
        {"role": "user", "content": example_fundus_content},
    ],
    response_format=FundusParameters,
)

In [32]:
json_obj = json.loads(completion.choices[0].message.content)
json_obj

{'is_valid': True,
 'od_color': 'бледно-розовый',
 'od_monotone': True,
 'od_size': 'нормальный',
 'od_shape': 'правильная',
 'od_border': 'четкие',
 'od_excavation_size': 'нормальный',
 'od_excavation_location': 'в центре',
 'od_excavation_ratio': 'None',
 'od_vessels_location': 'в центре',
 'od_pathology': 'None',
 'vessels_art_course': 'смещение височное',
 'vessels_art_turtuosity': 'нормальная',
 'vessels_art_bifurcation': 'нормальная',
 'vessels_art_caliber': 'нормальный',
 'vessels_vein_course': 'смещение височное',
 'vessels_vein_turtuosity': 'нормальная',
 'vessels_vein_bifurcation': 'нормальная',
 'vessels_vein_caliber': 'нормальный',
 'vessels_ratio': 'None',
 'vessels_pathology': 'None',
 'macula_macular_reflex': 'сглаженный',
 'macula_foveal_reflex': 'сглаженный',
 'macula_pathology': 'None',
 'peripheral_pathology': 'Ангиоматозный узел с преретинальным фиброзом, отграниченный лазеркоагулятами на 6-7 часах.',
 'other_diagnosis': 'None',
 'other_notes': 'None'}

И пример где их выделить нельзя.

В данном случае, значение параметра is_valid выставлено в False, а все остальные значения имеют параметр None

In [54]:
example_fundus_content = "не офтальмоскопируется из-за помутнения стекловидного тела."

completion = client.beta.chat.completions.parse(
    model=model_str,
    messages=[
        {"role": "system", "content": dedent(system_prompt_rus)},
        {"role": "user", "content": example_fundus_content},
    ],
    response_format=FundusParameters,
)
json_obj = json.loads(completion.choices[0].message.content)
json_obj

{'is_valid': False,
 'od_color': 'None',
 'od_monotone': False,
 'od_size': 'None',
 'od_shape': 'None',
 'od_border': 'None',
 'od_excavation_size': 'None',
 'od_excavation_location': 'None',
 'od_excavation_ratio': 'None',
 'od_vessels_location': 'None',
 'od_pathology': 'None',
 'vessels_art_course': 'None',
 'vessels_art_turtuosity': 'None',
 'vessels_art_bifurcation': 'None',
 'vessels_art_caliber': 'None',
 'vessels_vein_course': 'None',
 'vessels_vein_turtuosity': 'None',
 'vessels_vein_bifurcation': 'None',
 'vessels_vein_caliber': 'None',
 'vessels_ratio': 'None',
 'vessels_pathology': 'None',
 'macula_macular_reflex': 'None',
 'macula_foveal_reflex': 'None',
 'macula_pathology': 'None',
 'peripheral_pathology': 'None',
 'other_diagnosis': 'None',
 'other_notes': 'None'}

Реализуем функционал, представленный выше, в рамках функции, в который мы также будет добавлять к нашему объекту исходный текст описания глазного дна.

Данный текст мы потом сможем использовать в качестве целевой переменной при настройке (дообучении?) LLM 

In [26]:
def extract_entities(fundus_description, system_prompt):
    model_str="gpt-4o-2024-11-20"
    
    completion = client.beta.chat.completions.parse(
        model=model_str,
        messages=[
            {"role": "system", "content": dedent(system_prompt)},
            {"role": "user", "content": fundus_description},
        ],
        response_format=FundusParameters,
    )
    
    json_obj = json.loads(completion.choices[0].message.content)
    
    for param in json_obj:
        if param != 'is_valid' and 'None' in json_obj[param]:
            json_obj[param] = None
    
    json_obj['text'] = fundus_description
    
    return json_obj

In [94]:
filepath = r"D:\OneDrive\University\4_Year\DIPLOM\fundus_descriptions_529.txt"
json_array = []

with open(filepath, encoding="utf-8") as file:
    for line in tqdm(file, desc="Processing lines", unit="line"):
        json_obj = extract_entities(line.rstrip())
        json_array.append(json_obj)

Processing lines: 492line [43:15,  5.28s/line]


In [95]:
len(json_array)

492

In [96]:
import pandas as pd
df = pd.DataFrame(json_array)
df

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,...,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
0,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,нормальный,нормальный,None,нет очаговой патологии,None,None,"ДЗН бледно-розовый, с четкими границами, ход и..."
1,True,розовый,True,нормальный,правильная,четкие,нормальный,в центре,0.5,в центре,...,нормальный,None,None,нормальный,нормальный,None,None,None,None,"ДЗН - розовый, границы четкие, центральная экс..."
2,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,нормальный,нормальный,None,зоны дистрофии с эпиретинальным фиброзом,сетчатка источена,None,"ДЗН бледно-розовый, границы четкие, миопически..."
3,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,гладкий,отсутствует,None,Сетчатка слабо пигментирована по всему глазном...,None,На периферии без очаговой патологии,"ДЗН бледно-розовый, монотонный, границы четкие..."
4,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,нормальный,нормальный,"высокий ретиношизис, отграниченный коагулятами","высокий ретиношизис, отграниченный коагулятами",None,None,"ДЗН бледно-розовый, границы четкие, ход и кали..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,True,бледный,True,меньше нормы,правильная,четкие,нормальный,в центре,None,в центре,...,полнокровны,None,None,гладкий,гладкий,None,без очаговой патологии,None,None,"ДЗН бледный, монотоный, уменьшен в диаметре, г..."
488,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,нормальный,нормальный,None,None,None,None,"ДЗН бледно-розовый, границы четкие, сосуды не ..."
489,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,None,в центре,...,нормальный,None,None,нормальный,нормальный,воронкообразная отслойка сетчатки,None,открытая в переднем отделе,None,"воронкообразная отслойка сетчатки, открытая в ..."
490,True,бледноват,True,нормальный,правильная,нечеткие,нормальный,в центре,,в центре,...,расширенный,,Сосуды окружены экссудатом.,нормальный,нормальный,Субретинальный экссудат и сосудистые аневризмы...,Преретинальный фиброз и экссудативная отслойка...,Сосудистые аневризмы и экссудативные процессы.,Диск зрительного нерва монотонный с нечеткими ...,"ДЗН бледноват, монотонный, границы нечеткие, о..."


In [97]:
df.to_csv(r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\lines_529_1020.csv", index=False)   

In [99]:
df_1 = pd.read_csv(r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\lines_0_528.csv")
df_2 = pd.read_csv(r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\lines_529_1020.csv")

combined_df = pd.concat([df_1, df_2], ignore_index=True)
path = r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\fundus_descriptions_1020.csv"
combined_df.to_csv(path, index=False)

In [100]:
combined_df.shape

(1020, 28)

In [114]:
path = r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\fundus_descriptions_1020.csv"

combined_df = pd.read_csv(path)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   is_valid                  1020 non-null   bool  
 1   od_color                  904 non-null    object
 2   od_monotone               1020 non-null   bool  
 3   od_size                   904 non-null    object
 4   od_shape                  904 non-null    object
 5   od_border                 904 non-null    object
 6   od_excavation_size        903 non-null    object
 7   od_excavation_location    903 non-null    object
 8   od_excavation_ratio       53 non-null     object
 9   od_vessels_location       903 non-null    object
 10  od_pathology              319 non-null    object
 11  vessels_art_course        904 non-null    object
 12  vessels_art_turtuosity    903 non-null    object
 13  vessels_art_bifurcation   903 non-null    object
 14  vessels_art_caliber     

In [ ]:
for column in combined_df.columns:
    if column == 'text':
        continue
    display(combined_df[column].value_counts())
    print('='*100)

In [19]:
system_prompt_None = """
Извлеки параметры изображения глазного дна по его описанию. Если значение неизвестно (не указано в описании), то присвой значение None. Очень важно правильно написать None, не :None или как-то ещё, а точное значение None.

Если проводить офтольмоскопию невозможно, то укажи значение None для всех параметров, кроме is_valid.
Отслойка сетчатки не считается случаем, когда можно выделять параметры. 

Описание параметров:
- is_valid: Возможно ли визуализировать и проводить офтольмоскопию.
- od_color: Цвет диска зрительного нерва, который может указывать на различные состояния его здоровья (например, серый, бледный или белый).
- od_monotone: Булевый параметр, указывающий, имеет ли диск зрительного нерва монотонный (однородный) цвет. 
- od_size: Размер диска зрительного нерва (например, нормальный, больше нормы или меньше нормы), что может помочь выявить отклонения.
- od_shape: Форма диска зрительного нерва (например, правильная, овальная или неправильная).
- od_border: Чёткость границ диска зрительного нерва (например, четкие или размытые), важный показатель для диагностики патологий.
- od_excavation_size: Размер экскавации (углубления) на диске зрительного нерва (например, нормальный, больше нормы или меньше нормы), который может варьироваться в зависимости от таких состояний, как глаукома. 
- od_excavation_location: Расположение экскавации на диске зрительного нерва (например, в центре, верхний, нижний, наружный, внутренний, верхне-наружный, верхне-внутренний, нижне-наружный, нижне-внутренний).
- od_excavation_ratio: Отношение размеров экскавации к диску (CDR), ключевой показатель для оценки риска глаукомы.
- od_vessels_location: Расположение сосудистого пучка на диске зрительного нерва (например, в центре, верхний, нижний, наружный, внутренний, верхне-наружный, верхне-внутренний, нижне-наружный, нижне-внутренний). 
- od_pathology: Свободный текст, описывающий патологию диска зрительного нерва.
- vessels_art_course: Направление или смещение артерий (например, нормальное, смещение наружу или смещение внутрь). 
- vessels_art_turtuosity: Извитость артерий (например, нормальная, извитая или прямая), отражающая состояние сосудов.
- vessels_art_bifurcation: Угол бифуркации артерий (например, нормальная, под острым углом или под тупым углом), важный для оценки сосудов.
- vessels_art_caliber: Калибр (ширина) артерий (например, нормальный, расширенный или суженный), который может указывать на проблемы с кровообращением.
- vessels_vein_course: Направление или смещение вен (например, нормальное, наружное или внутреннее смещение).
- vessels_vein_turtuosity: Извитость вен (например, нормальная, извитая или прямая), предоставляющая информацию о системных или глазных заболеваниях.
- vessels_vein_bifurcation: Угол бифуркации вен (например, нормальная, под острым углом или под тупым углом), часто изучается для оценки сосудистых аномалий.
- vessels_vein_caliber: Калибр (ширина) вен (например, нормальный, расширенный или суженный), важный для диагностики сосудистых патологий.
- vessels_ratio: Свободный текст, указывающий соотношение артерий и вен.
- vessels_pathology: Свободный текст, описывающий патологию сосудов.
- macula_macular_reflex: Состояние макулярного рефлекса, который может быть нормальным, гладким или отсутствовать, предоставляя информацию о здоровье макулы.
- macula_foveal_reflex: Состояние фовеального рефлекса (например, нормальный, гладкий или отсутствует), критично для оценки качества центрального зрения.
- macula_pathology: Свободный текст, описывающий патологию макулы.
- peripheral_pathology: Свободный текст, описывающий патологию периферии глазного дна.
- other_diagnosis: Свободный текст, указывающий диагноз.
- other_notes: Свободный текст, указывающий примечания.
"""

In [30]:
filepath = r"D:\OneDrive\University\4_Year\DIPLOM\fundus_descr_clear.txt"
json_array = []

with open(filepath, encoding="utf-8") as file:
    for line in tqdm(file, desc="Processing lines", unit="line"):
        json_obj = extract_entities(line.rstrip(), system_prompt_None)
        json_array.append(json_obj)

Processing lines: 891line [55:25,  3.73s/line]


In [31]:
import pandas as pd
df = pd.DataFrame(json_array)
df

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,...,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
0,True,бледно-розовый,None,None,None,четкие,None,None,None,None,...,нормальный,None,None,четкий,None,слегка гиперпигментирована,участки разряжения пигмента,альбинотическое глазное дно,None,"ДЗН бледно-розовый, границы четкие, ход и кали..."
1,True,/Events,/Events,/Events,/Events,размытые,None,None,None,None,...,None,None,MistVenin Varie rise>: отличистрерство,ums,/Refle,None,Normative!,Donfuraxe,Sons,"За флером: границы ДЗН нечеткие, сосуды расшир..."
2,True,бледно-розовый,None,None,None,четкие,None,None,None,None,...,не изменен,None,None,ровный,None,None,без патологии,None,None,"ДЗН бледно-розовый, с четкими границами, ход и..."
3,True,бледно-розовый,None,None,None,четкие,None,None,None,в височную сторону,...,None,None,None,сглажены,None,None,"аваскулярная зона, локальная отслойка сетчатки...",None,None,"ДЗН бледно-розовый, границы четкие, артерии не..."
4,True,бледный,None,гипопластичен,None,None,None,None,None,None,...,None,None,None,None,None,None,Описывается аваскулярная сетчатка с перераспре...,None,None,"ДЗН бледный, гипопластичен, тракционно деформи..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,True,бледный,true,уменьшен в диаметре,None,четкие,None,None,None,None,...,полнокровны,None,None,сглаженный,сглаженный,None,без очаговой патологии,None,None,"ДЗН бледный, монотоный, уменьшен в диаметре, г..."
887,True,бледно-розовый,None,None,None,четкие,None,None,None,None,...,нормальный,None,None,выражены,выражены,None,отсутствует,None,None,"ДЗН бледно-розовый, границы четкие, сосуды не ..."
888,False,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"воронкообразная отслойка сетчатки, открытая в ..."
889,False,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"ДЗН бледноват, монотонный, границы нечеткие, о..."


In [33]:
df.to_csv(r"D:\OneDrive\University\4_Year\DIPLOM\csv_s\df_none2.csv", index=False)   